In [1]:
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol
# Import date and time zone libraries
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def initialize(context):
    dji = [
            "AAPL",
            "AXP",
            "BA",
            "CAT",
            "CSCO",
            "CVX",
            "DIS",
            "DWDP",
            "GS",
            "HD",
            "IBM",
            "INTC",
            "JNJ",
            "JPM",
            "KO",
            "MCD",
            "MMM",
            "MRK",
            "MSFT",
            "NKE",
            "PFE",
            "PG",
            "TRV",
            "UNH",
            "UTX",
            "V",
            "VZ",
            "WBA",
            "WMT",
            "XOM",  
    ]
    context.dji_symbols = [symbol(s) for s in dji]
    # Moving average window
    context.index_average_window = 100

                
def handle_data(context, data):
    stock_hist = data.history(context.dji_symbols, "close", context.index_average_window, "1d")
    stock_analytics = pd.DataFrame()
    stock_analytics['above_mean'] = stock_hist.iloc[-1] > stock_hist.mean()

    stock_analytics.loc[stock_analytics['above_mean'] == True, 'weight'] = 1/len(context.dji_symbols)
    
    # Set weight to zero for the rest
    stock_analytics.loc[stock_analytics['above_mean'] == False, 'weight'] = 0.0
    for stock, analytics in stock_analytics.iterrows():
        # Check if the stock can be traded
        if data.can_trade(stock):
            # Place the trade
            order_target_percent(stock, analytics['weight'])
            
def analyze(context, perf):
    fig = plt.figure(figsize=(12, 8))
    
    # First chart
    ax = fig.add_subplot(311)
    ax.set_title('Strategy Results')
    ax.plot(perf['portfolio_value'], linestyle='-', 
                label='Equity Curve', linewidth=3.0)
    ax.legend()
    ax.grid(False)
    
    # Second chart
    ax = fig.add_subplot(312)
    ax.plot(perf['gross_leverage'], 
            label='Exposure', linestyle='-', linewidth=1.0)
    ax.legend()
    ax.grid(True)

    # Third chart
    ax = fig.add_subplot(313)
    ax.plot(perf['returns'], label='Returns', linestyle='-.', linewidth=1.0)
    ax.legend()
    ax.grid(True)

In [ ]:
start_date = pd.Timestamp('1997-01-01')
end_date = pd.Timestamp('2018-03-27')
results = run_algorithm(
    start = start_date.tz_localize(None),
    end = end_date.tz_localize(None),
    initialize=initialize,
    analyze=analyze,
    handle_data=handle_data,
    capital_base=10000,
    data_frequency='daily',
    bundle='quandl'
)